В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [2]:
# Load the data
raw_df = pd.read_csv('train.csv')

# Define target column and input columns
target_col = 'Exited'
input_cols = [col for col in raw_df.columns if col not in [target_col, 'id']]

# Split data into training and validation sets
train_df, val_df = split_train_val(raw_df, target_col)

# Separate inputs and targets for training and validation sets
train_inputs, train_targets = separate_inputs_targets(train_df, input_cols, target_col)
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols, target_col)

print("Data loaded and split successfully.")
print("Training inputs shape:", train_inputs.shape)
print("Training targets shape:", train_targets.shape)
print("Validation inputs shape:", val_inputs.shape)
print("Validation targets shape:", val_targets.shape)

Data loaded and split successfully.
Training inputs shape: (12000, 12)
Training targets shape: (12000,)
Validation inputs shape: (3000, 12)
Validation targets shape: (3000,)


1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [3]:
categorical_cols = train_inputs.select_dtypes(include='object').columns

for col in categorical_cols:
    train_inputs[col] = train_inputs[col].astype('category')
    val_inputs[col] = val_inputs[col].astype('category')

print("Categorical features converted successfully.")
print("Data types in train_inputs after conversion:")
print(train_inputs.dtypes)
print("\nData types in val_inputs after conversion:")
print(val_inputs.dtypes)

Categorical features converted successfully.
Data types in train_inputs after conversion:
CustomerId          float64
Surname            category
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
dtype: object

Data types in val_inputs after conversion:
CustomerId          float64
Surname            category
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
dtype: object


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [4]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Initialize and train the XGBoostClassifier model
# Use handle_missing='enable' to handle missing values
# Use enable_categorical=True to handle categorical features
# Consider adding device='cuda' if you have a GPU
xgb_clf = xgb.XGBClassifier(objective='binary:logistic',
                            eval_metric='auc',
                            use_label_encoder=False, # Deprecated, set to False
                            n_estimators=100,
                            learning_rate=0.1,
                            max_depth=5,
                            random_state=42,
                            n_jobs=-1,
                            tree_method='hist', # Use hist for better performance with categorical features
                            enable_categorical=True,
                            handle_missing='enable')

xgb_clf.fit(train_inputs, train_targets)

# Predict probabilities
train_pred_proba_xgb = xgb_clf.predict_proba(train_inputs)[:, 1]
val_pred_proba_xgb = xgb_clf.predict_proba(val_inputs)[:, 1]

# Calculate AUROC
train_roc_auc_xgb = roc_auc_score(train_targets, train_pred_proba_xgb)
val_roc_auc_xgb = roc_auc_score(val_targets, val_pred_proba_xgb)

print(f"AUROC on training set (XGBoost): {train_roc_auc_xgb:.4f}")
print(f"AUROC on validation set (XGBoost): {val_roc_auc_xgb:.4f}")

# Conclusion about the model
print("\nВисновки щодо моделі XGBoostClassifier:")
if train_roc_auc_xgb > val_roc_auc_xgb and (train_roc_auc_xgb - val_roc_auc_xgb) > 0.05:
    print("Модель, можливо, має високу дисперсію (high variance), оскільки якість на тренувальному наборі значно вища, ніж на валідаційному.")
elif train_roc_auc_xgb < 0.7 or val_roc_auc_xgb < 0.7: # Example threshold, adjust as needed
     print("Модель, можливо, має високе зміщення (high bias), оскільки якість на обох наборах невисока.")
else:
    print("Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.")

# Placeholder for comparison with Decision Trees (assuming previous results are available)
print("\nПорівняння з Decision Trees (потрібно порівняти з результатами попередніх завдань):")
print("Для повного порівняння необхідно мати результати AUROC з моделі Decision Trees.")
print("Якщо AUROC на валідаційному наборі вищий за Decision Trees, то якість покращилась.")

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:05:08] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "handle_missing", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUROC on training set (XGBoost): 0.9788
AUROC on validation set (XGBoost): 0.9323

Висновки щодо моделі XGBoostClassifier:
Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.

Порівняння з Decision Trees (потрібно порівняти з результатами попередніх завдань):
Для повного порівняння необхідно мати результати AUROC з моделі Decision Trees.
Якщо AUROC на валідаційному наборі вищий за Decision Trees, то якість покращилась.


3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import xgboost as xgb

# 1. Define the objective function
def objective(params):
    """
    Objective function for Hyperopt to minimize (negative AUROC).
    """
    # Need to convert certain parameters to int within the objective function
    # as they are sampled as floats by hyperopt
    params_int = {k: int(v) if k in ['n_estimators', 'max_depth', 'min_child_weight'] else v for k, v in params.items()}


    clf = xgb.XGBClassifier(objective='binary:logistic',
                            eval_metric='auc',
                            use_label_encoder=False,
                            random_state=42,
                            n_jobs=-1,
                            tree_method='hist', # Use hist for better performance with categorical features
                            enable_categorical=True,
                            handle_missing='enable', # This parameter might be ignored in newer versions
                            **params_int)

    clf.fit(train_inputs, train_targets)

    val_pred_proba = clf.predict_proba(val_inputs)[:, 1]
    roc_auc = roc_auc_score(val_targets, val_pred_proba)

    # Hyperopt minimizes the objective function, so we return the negative AUROC
    return {'loss': -roc_auc, 'status': STATUS_OK}

# 2. Define the hyperparameter space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # From 0.05 to 1 (~0.05 to 1)
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, 0), # From exp(-5) to exp(0) (~0.0067 to 1)
    'reg_lambda': hp.loguniform('reg_lambda', -5, 0) # From exp(-5) to exp(0) (~0.0067 to 1)
}


# 3. Run the optimization
trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=tpe.suggest,
                   max_evals=20, # Set the number of rounds to 20 as requested
                   trials=trials)

print("\nНайкращі гіперпараметри:", best_params)

# 4. Train the final model with the best hyperparameters
# Need to convert best_params from float to int for certain parameters
best_params_final = {k: int(v) if k in ['n_estimators', 'max_depth', 'min_child_weight'] else v for k, v in best_params.items()}


final_clf = xgb.XGBClassifier(objective='binary:logistic',
                              eval_metric='auc',
                              use_label_encoder=False,
                              random_state=42,
                              n_jobs=-1,
                              tree_method='hist',
                              enable_categorical=True,
                              handle_missing='enable', # This parameter might be ignored in newer versions
                              **best_params_final)

final_clf.fit(train_inputs, train_targets)

# 5. Evaluate the final model
train_pred_proba_final = final_clf.predict_proba(train_inputs)[:, 1]
val_pred_proba_final = final_clf.predict_proba(val_inputs)[:, 1]

train_roc_auc_final = roc_auc_score(train_targets, train_pred_proba_final)
val_roc_auc_final = roc_auc_score(val_targets, val_pred_proba_final)

print(f"\nAUROC on training set (XGBoost with Hyperopt): {train_roc_auc_final:.4f}")
print(f"AUROC on validation set (XGBoost with Hyperopt): {val_roc_auc_final:.4f}")

# Conclusion about the model
print("\nВисновки щодо моделі XGBoostClassifier з Hyperopt:")
if train_roc_auc_final > val_roc_auc_final and (train_roc_auc_final - val_roc_auc_final) > 0.05:
    print("Модель, можливо, має високу дисперсію (high variance), оскільки якість на тренувальному наборі значно вища, ніж на валідаційному.")
elif train_roc_auc_final < 0.7 or val_roc_auc_final < 0.7: # Example threshold, adjust as needed
     print("Модель, можливо, має високе зміщення (high bias), оскільки якість на обох наборах невисока.")
else:
    print("Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.")

# Comparison with the previous model
print("\nПорівняння з попередньою моделлю XGBoostClassifier (без Hyperopt):")
# Ensure val_roc_auc_xgb is defined before using it
if 'val_roc_auc_xgb' in globals():
    print(f"AUROC на валідаційному наборі без Hyperopt: {val_roc_auc_xgb:.4f}")
    print(f"AUROC на валідаційному наборі з Hyperopt: {val_roc_auc_final:.4f}")

    if val_roc_auc_final > val_roc_auc_xgb:
        print("Якість моделі покращилась після оптимізації гіперпараметрів за допомогою Hyperopt.")
    elif val_roc_auc_final < val_roc_auc_xgb:
        print("Якість моделі погіршилась після оптимізації гіперпараметрів за допомогою Hyperopt.")
    else:
        print("Якість моделі залишилась без змін після оптимізації гіперпараметрів за допомогою Hyperopt.")
else:
    print("Результати AUROC без Hyperopt недоступні для порівняння.")

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:08:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "handle_missing", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



  5%|▌         | 1/20 [00:00<00:15,  1.24trial/s, best loss: -0.9297798202894573]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:08:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "handle_missing", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



 10%|█         | 2/20 [00:01<00:08,  2.07trial/s, best loss: -0.9364565470882776]

...

 95%|█████████▌| 19/20 [00:13<00:00,  1.26trial/s, best loss: -0.9364565470882776]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:08:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "handle_missing", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



100%|██████████| 20/20 [00:14<00:00,  1.42trial/s, best loss: -0.9364565470882776]

Найкращі гіперпараметри: {'colsample_bytree': np.float64(0.7492150433815387), 'gamma': np.float64(0.46920665998710176), 'learning_rate': np.float64(0.05583909944478992), 'max_depth': np.float64(3.0), 'min_child_weight': np.float64(4.0), 'n_estimators': np.float64(200.0), 'reg_alpha': np.float64(0.008534347278250896), 'reg_lambda': np.float64(0.6038030861691353), 'subsample': np.float64(0.9174714583233008)}


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:08:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "handle_missing", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



AUROC on training set (XGBoost with Hyperopt): 0.9473
AUROC on validation set (XGBoost with Hyperopt): 0.9365

Висновки щодо моделі XGBoostClassifier з Hyperopt:
Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.

Порівняння з попередньою моделлю XGBoostClassifier (без Hyperopt):
AUROC на валідаційному наборі без Hyperopt: 0.9323
AUROC на валідаційному наборі з Hyperopt: 0.9365
Якість моделі покращилась після оптимізації гіперпараметрів за допомогою Hyperopt.


4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [10]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Identify categorical feature indices for LightGBM
categorical_feature_names = train_inputs.select_dtypes(include='category').columns
categorical_feature_indices = [train_inputs.columns.get_loc(col) for col in categorical_feature_names]

# Initialize and train the LightGBM model
# LightGBM handles missing values and categorical features internally with appropriate settings
lgb_clf = lgb.LGBMClassifier(objective='binary',
                             metric='auc',
                             n_estimators=100,
                             learning_rate=0.1,
                             max_depth=5,
                             random_state=42,
                             n_jobs=-1,
                             # Use categorical_feature parameter to specify categorical columns by index
                             categorical_feature=categorical_feature_indices
                            )

# LightGBM can train directly on pandas DataFrames with categorical dtypes
lgb_clf.fit(train_inputs, train_targets)

# Predict probabilities
train_pred_proba_lgb = lgb_clf.predict_proba(train_inputs)[:, 1]
val_pred_proba_lgb = lgb_clf.predict_proba(val_inputs)[:, 1]

# Calculate AUROC
train_roc_auc_lgb = roc_auc_score(train_targets, train_pred_proba_lgb)
val_roc_auc_lgb = roc_auc_score(val_targets, val_pred_proba_lgb)

print(f"AUROC on training set (LightGBM): {train_roc_auc_lgb:.4f}")
print(f"AUROC on validation set (LightGBM): {val_roc_auc_lgb:.4f}")

# Conclusion about the model
print("\nВисновки щодо моделі LightGBM:")
if train_roc_auc_lgb > val_roc_auc_lgb and (train_roc_auc_lgb - val_roc_auc_lgb) > 0.05:
    print("Модель, можливо, має високу дисперсію (high variance), оскільки якість на тренувальному наборі значно вища, ніж на валідаційному.")
elif train_roc_auc_lgb < 0.7 or val_roc_auc_lgb < 0.7: # Example threshold, adjust as needed
     print("Модель, можливо, має високе зміщення (high bias), оскільки якість на обох наборах невисока.")
else:
    print("Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.")

# Comparison with the XGBoost model
print("\nПорівняння з моделлю XGBoostClassifier (без Hyperopt):")
# Ensure val_roc_auc_xgb is defined before using it
if 'val_roc_auc_xgb' in globals():
    print(f"AUROC на валідаційному наборі XGBoost (без Hyperopt): {val_roc_auc_xgb:.4f}")
    print(f"AUROC на валідаційному наборі LightGBM: {val_roc_auc_lgb:.4f}")

    if val_roc_auc_lgb > val_roc_auc_xgb:
        print("Якість моделі LightGBM краща за початкову модель XGBoost.")
    elif val_roc_auc_lgb < val_roc_auc_xgb:
        print("Якість моделі LightGBM гірша за початкову модель XGBoost.")
    else:
        print("Якість моделей LightGBM та початкової XGBoost схожа.")
else:
    print("Результати AUROC для початкової моделі XGBoost недоступні для порівняння.")

print("\nПорівняння з оптимізованою моделлю XGBoostClassifier (з Hyperopt):")
# Ensure val_roc_auc_final is defined before using it
if 'val_roc_auc_final' in globals():
    print(f"AUROC на валідаційному наборі XGBoost (з Hyperopt): {val_roc_auc_final:.4f}")
    print(f"AUROC на валідаційному наборі LightGBM: {val_roc_auc_lgb:.4f}")

    if val_roc_auc_lgb > val_roc_auc_final:
        print("Якість моделі LightGBM краща за оптимізовану модель XGBoost.")
    elif val_roc_auc_lgb < val_roc_auc_final:
        print("Якість моделі LightGBM гірша за оптимізовану модель XGBoost.")
    else:
        print("Якість моделей LightGBM та оптимізованої XGBoost схожа.")
else:
    print("Результати AUROC для оптимізованої моделі XGBoost недоступні для порівняння.")

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=1,3,4, categorical_column=1,3,4 will be ignored. Current value: categorical_feature=1,3,4
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1571
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# Identify categorical feature indices for LightGBM (if not already done)
categorical_feature_names = train_inputs.select_dtypes(include='category').columns
categorical_feature_indices = [train_inputs.columns.get_loc(col) for col in categorical_feature_names]


# 1. Define the objective function for LightGBM
def objective_lgb(params):
    """
    Objective function for Hyperopt to minimize (negative AUROC) for LightGBM.
    """
    # Need to convert certain parameters to int within the objective function
    params_int = {k: int(v) if k in ['n_estimators', 'max_depth', 'num_leaves', 'min_child_samples'] else v for k, v in params.items()}

    clf = lgb.LGBMClassifier(objective='binary',
                             metric='auc',
                             random_state=42,
                             n_jobs=-1,
                             categorical_feature=categorical_feature_indices,
                             **params_int)

    clf.fit(train_inputs, train_targets)

    val_pred_proba = clf.predict_proba(val_inputs)[:, 1]
    roc_auc = roc_auc_score(val_targets, val_pred_proba)

    # Hyperopt minimizes the objective function, so we return the negative AUROC
    return {'loss': -roc_auc, 'status': STATUS_OK}

# 2. Define the hyperparameter space for LightGBM
space_lgb = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # From 0.05 to 1
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 60, 5), # Often between 20 and 60 for LightGBM
    'min_child_samples': hp.quniform('min_child_samples', 5, 50, 5),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, 0),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 0)
}


# 3. Run the optimization for LightGBM
trials_lgb = Trials()
best_params_lgb = fmin(fn=objective_lgb,
                       space=space_lgb,
                       algo=tpe.suggest,
                       max_evals=10, # Set the number of rounds to 10 as requested
                       trials=trials_lgb)

print("\nНайкращі гіперпараметри для LightGBM:", best_params_lgb)

# 4. Train the final LightGBM model with the best hyperparameters
# Need to convert best_params from float to int for certain parameters
best_params_lgb_final = {k: int(v) if k in ['n_estimators', 'max_depth', 'num_leaves', 'min_child_samples'] else v for k, v in best_params_lgb.items()}


final_lgb_clf = lgb.LGBMClassifier(objective='binary',
                                   metric='auc',
                                   random_state=42,
                                   n_jobs=-1,
                                   categorical_feature=categorical_feature_indices,
                                   **best_params_lgb_final)

final_lgb_clf.fit(train_inputs, train_targets)

# 5. Evaluate the final LightGBM model
train_pred_proba_final_lgb = final_lgb_clf.predict_proba(train_inputs)[:, 1]
val_pred_proba_final_lgb = final_lgb_clf.predict_proba(val_inputs)[:, 1]

train_roc_auc_final_lgb = roc_auc_score(train_targets, train_pred_proba_final_lgb)
val_roc_auc_final_lgb = roc_auc_score(val_targets, val_pred_proba_final_lgb)

print(f"\nAUROC on training set (LightGBM with Hyperopt): {train_roc_auc_final_lgb:.4f}")
print(f"AUROC on validation set (LightGBM with Hyperopt): {val_roc_auc_final_lgb:.4f}")

# Conclusion about the model
print("\nВисновки щодо моделі LightGBM з Hyperopt:")
if train_roc_auc_final_lgb > val_roc_auc_final_lgb and (train_roc_auc_final_lgb - val_roc_auc_final_lgb) > 0.05:
    print("Модель, можливо, має високу дисперсію (high variance), оскільки якість на тренувальному наборі значно вища, ніж на валідаційному.")
elif val_roc_auc_final_lgb < 0.7: # Example threshold, adjust as needed
     print("Модель, можливо, має високе зміщення (high bias), оскільки якість на валідаційному наборі невисока.")
else:
    print("Модель показує схожу якість на тренувальному та валідаційному наборах, що свідчить про збалансоване співвідношення зміщення та дисперсії.")

# Comparison with previous models
print("\nПорівняння з попередніми моделями:")
# Ensure previous AUROC values are defined before using them
if 'val_roc_auc_xgb' in globals():
    print(f"AUROC на валідаційному наборі XGBoost (без Hyperopt): {val_roc_auc_xgb:.4f}")
if 'val_roc_auc_final' in globals():
    print(f"AUROC на валідаційному наборі XGBoost (з Hyperopt): {val_roc_auc_final:.4f}")
if 'val_roc_auc_lgb' in globals():
     print(f"AUROC на валідаційному наборі LightGBM (без Hyperopt): {val_roc_auc_lgb:.4f}")

print(f"AUROC на валідаційному наборі LightGBM (з Hyperopt): {val_roc_auc_final_lgb:.4f}")

# Determine which model performed best on the validation set
best_val_auc = max(val_roc_auc_xgb if 'val_roc_auc_xgb' in globals() else 0,
                   val_roc_auc_final if 'val_roc_auc_final' in globals() else 0,
                   val_roc_auc_lgb if 'val_roc_auc_lgb' in globals() else 0,
                   val_roc_auc_final_lgb)

if val_roc_auc_final_lgb == best_val_auc:
    print("\nОптимізована модель LightGBM показала найкращий результат на валідаційному наборі.")
elif val_roc_auc_final == best_val_auc:
    print("\nОптимізована модель XGBoost показала найкращий результат на валідаційному наборі.")
elif val_roc_auc_lgb == best_val_auc:
    print("\nПочаткова модель LightGBM показала найкращий результат на валідаційному наборі.")
elif val_roc_auc_xgb == best_val_auc:
     print("\nПочаткова модель XGBoost показала найкращий результат на валідаційному наборі.")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=1,3,4, categorical_column=1,3,4 will be ignored. Current value: categorical_feature=1,3,4
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1571
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=1,3,4, categorical_column=1,3,4 will be ignored. Current value: categorical_feature=1,3,4
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1571
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
 20%|██        | 2/10 [00:03<00:13,  1.63s/trial, best loss: -0.9297201454146375]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



AUROC on training set (LightGBM with Hyperopt): 0.9920
AUROC on validation set (LightGBM with Hyperopt): 0.9317

Висновки щодо моделі LightGBM з Hyperopt:
Модель, можливо, має високу дисперсію (high variance), оскільки якість на тренувальному наборі значно вища, ніж на валідаційному.

Порівняння з попередніми моделями:
AUROC на валідаційному наборі XGBoost (без Hyperopt): 0.9323
AUROC на валідаційному наборі XGBoost (з Hyperopt): 0.9365
AUROC на валідаційному наборі LightGBM (без Hyperopt): 0.9323
AUROC на валідаційному наборі LightGBM (з Hyperopt): 0.9317

Оптимізована модель XGBoost показала найкращий результат на валідаційному наборі.


6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [13]:
# Load the test data
try:
    test_df = pd.read_csv('test.csv')
except FileNotFoundError:
    print("Error: test.csv not found. Please make sure the file is in the correct directory.")
    test_df = None

if test_df is not None:
    # Prepare test data: select input columns and convert categorical features
    test_inputs = test_df[input_cols].copy()

    for col in categorical_cols:
         if col in test_inputs.columns:
              test_inputs[col] = test_inputs[col].astype('category')


    # Make predictions on the test data using the best model (optimized XGBoost)
    # Ensure the best model variable name matches the one from the previous step
    # In this case, it's 'final_clf' from cell WhR1g9B4433r
    if 'final_clf' in globals():
        test_predictions = final_clf.predict_proba(test_inputs)[:, 1]

        # Create the submission DataFrame
        submission_df = pd.DataFrame({'id': test_df['id'], 'Exited': test_predictions})

        # Save the submission file
        submission_df.to_csv('submission.csv', index=False)

        print("Submission file 'submission.csv' created successfully!")
        print(submission_df.head())
    else:
        print("Error: Optimized XGBoost model ('final_clf') not found. Please ensure the previous cell ran successfully.")

Submission file 'submission.csv' created successfully!
      id    Exited
0  15000  0.078740
1  15001  0.017630
2  15002  0.070709
3  15003  0.509270
4  15004  0.038327
